In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

In [ ]:
data = pd.DataFrame({
    'time': [2, 5, 10, 6, 8, 15, 20, 25, 30, 35],
    'event': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]  # 1 = event occurred, 0 = censored
})


In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(data['time'], event_observed=data['event'])


In [ ]:
time_points = kmf.survival_function_.index
survival_probs = kmf.survival_function_['KM_estimate']
log_log_survival = np.log(-np.log(survival_probs))
log_time = np.log(time_points)


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(log_time, log_log_survival, marker='o', linestyle='-', color='blue')
plt.title('Weibull Probability Plot (Log-Log of Survival vs. Log Time)')
plt.xlabel('log(T)')
plt.ylabel('log(-log(S(t)))')
plt.grid(True)
plt.show()
